In [ ]:
sm = snakemake

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import spherpro.bro as sb
import spherpro.db as db

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotnine as gg

import scanpy as sc
%matplotlib inline

In [ ]:
from src.variables import Vars
#from src.config import Conf

In [ ]:
def censor_dat(x, q=99.9):
    x = np.copy(x)
    pmax = np.percentile(x,q=q)
    x[ x > pmax ] = pmax
    pmin = np.percentile(x,q=100-q)
    x[x < pmin] = pmin
    return x

def cur_logtransf(x):
    return np.log10(x+0.1)

def cur_transf(x):
    x= censor_dat(x, 99.9)
    x= cur_logtransf(x)
    return x

## Aim

Cluster cell cylce and cell deadth markers to identify apoptotic and mitotic cells

In [ ]:
class V(Vars):
    COL_LEIDEN = 'leiden'
    COL_ISCC = 'is_cc'
    COL_METAL = 'metal'
    
class C:
    chans_dead = ['Yb172', 'Yb174']
    chan_phh3 = 'Eu153'
    fn_config = sm.input.fn_config
    


## 0) Setup configuration and bro

In [ ]:
bro = sb.get_bro(C.fn_config)

## 1) Analysis

In [ ]:
import spherpro.bromodules.helpers_vz as helpers_vz
hpr = helpers_vz.HelperVZ(bro)

In [ ]:
dat_pannel = bro.data.pannel

In [ ]:
dat_pannel.columns

In [ ]:
chan_cc = dat_pannel.query(f'{V.COL_ISCC} == True')[V.COL_METAL]

In [ ]:

fil = bro.filters.measurements.get_measmeta_filter_statements(
    channel_names=[tuple(list(chan_cc)+ C.chans_dead)],
    stack_names=['FullStackFiltered'],
    measurement_names=['MeanIntensityComp'],
    measurement_types=[None])

q_meas = (bro.data.get_measmeta_query()
          .filter(fil)
          .add_columns(db.ref_stacks.scale, db.ref_planes.channel_name,
                      db.stacks.stack_name)
         )

q_obj = (bro.data.get_objectmeta_query()
         .join(db.conditions, db.conditions.condition_id == db.images.condition_id)
         .filter(db.objects.object_type == 'cell')
         .add_entity(db.conditions)
         .add_entity(db.acquisitions)
         .add_entity(db.sites)
         .add_entity(db.slideacs)
         .add_entity(db.slides)
        )

In [ ]:
%%time
dat = bro.io.objmeasurements.get_measurements(q_meas=q_meas, q_obj=q_obj)
dat = bro.io.objmeasurements.scale_anndata(dat)

In [ ]:
# Check why the index isnt called index any more
dat.obs.index.name = 'index'

In [ ]:
dat.obs = dat.obs.loc[:,~dat.obs.columns.duplicated()]
dat.var = dat.var.loc[:,~dat.var.columns.duplicated()]

In [ ]:
dat.X = np.apply_along_axis(cur_transf, 0, dat.X)

Subset only cellcyle genes + , do clustering

In [ ]:
bro.data._read_experiment_layout()

In [ ]:
dat_d2rim = hpr.get_d2rim()

In [ ]:
bro.helpers.anndata.add_anndata_obsmeta(dat,dat_d2rim[[db.objects.object_id.key, V.COL_D2RIM]])

In [ ]:
bro.helpers.anndata.add_anndata_obsmeta(dat, bro.data.experiment_layout)

In [ ]:
bro.data.pannel[db.ref_planes.channel_name.key] = bro.data.pannel['metal']
bro.helpers.anndata.add_anndata_varmeta(dat, bro.data.pannel)

Cluster over all:

In [ ]:
sc.pp.neighbors(dat, use_rep='X',)

In [ ]:
sc.tl.umap(dat)

In [ ]:
sc.tl.leiden(dat)

Cluster by cell line:

In [ ]:
datf_cl_cc  = {cl: dat[dat.obs['cellline'] == cl,:] for cl in dat.obs['cellline'].unique()}

In [ ]:
for k, d in datf_cl_cc.items():
    sc.pp.neighbors(d, use_rep='X',)
    sc.tl.umap(d)
    sc.tl.leiden(d)

In [ ]:
for k, d in datf_cl_cc.items():
    print(k)
    sc.pl.umap(d,color=list(d.var.goodname)+['distrim'],gene_symbols='goodname')
    plt.gcf().suptitle(k)

In [ ]:
for k, d in datf_cl_cc.items():
    sc.pl.umap(d,color='distrim')
    plt.suptitle(k)

In [ ]:
for k, d in datf_cl_cc.items():
    sc.pl.umap(d,color='leiden')
    #plt.suptitle(k)

Cluster markers

In [ ]:
for k, d in datf_cl_cc.items():
    print(k)
    sc.tl.dendrogram(d, groupby='leiden')
    sc.pl.stacked_violin(d, var_names=d.var.goodname,groupby='leiden', swap_axes=True, dendrogram=True, gene_symbols='goodname')

Identify apoptotic clusters  
-> This can be done equally well with cleaved Parp and cleaved Caspase (see cluster markers)

In [ ]:
clust_dead = {}
tresh_pos = -0.4
apopt_channel = C.chans_dead[1]
apopt_name = bro.helpers.dbhelp.get_target_by_channel(apopt_channel)
for k, d in datf_cl_cc.items():
    d=(d.obs[[V.COL_LEIDEN]]
        .assign(**{V.COL_VALUE: d.X[:, d.var[V.COL_CHANNELNAME] == apopt_channel]})
        .groupby(V.COL_LEIDEN).median()

    )
    d.hist(bins=30)
    plt.title(f'{k} - {apopt_name}')
    plt.vlines(tresh_pos,0, 10)
    clust_dead[k] = [l for l in d.query(f'{V.COL_VALUE} > {tresh_pos}').index]
print(f'Apoptotic clusters: {clust_dead}')

Identify Mitotitic clusters

In [ ]:
clust_mitosis = {}
tresh_pos = -0.4
for k, d in datf_cl_cc.items():
    d=(d.obs[[V.COL_LEIDEN]]
        .assign(**{V.COL_VALUE: d.X[:, d.var[V.COL_CHANNELNAME] == C.chan_phh3]})
        .groupby(V.COL_LEIDEN).mean()

    )
    d.hist(bins=30)
    plt.title(k)
    plt.vlines(tresh_pos,0, 10)
    clust_mitosis[k] = [l for l in d.query(f'{V.COL_VALUE} > {tresh_pos}').index]
print(f'Mitotic clusters: {clust_mitosis}')

Get all mitosis cells for a 'is-mitosis' filter

In [ ]:
obs_mitosis = []
obs_dead = []
for k, d in datf_cl_cc.items():
    obs_mitosis.append(d.obs.index[d.obs['leiden']
                                 .isin(clust_mitosis[k])
                                  ].values)
    obs_dead.append(d.obs.index[d.obs['leiden']
                              .isin(clust_dead[k])
                               ].values
                   )
obs_mitosis = np.concatenate(obs_mitosis)
obs_dead = np.concatenate(obs_dead)

In [ ]:
dat.obs['mitosis'] = dat.obs.index.isin(obs_mitosis)
dat.obs['dead'] = dat.obs.index.isin(obs_dead)

In [ ]:
dat.obs[db.object_filters.object_id.key] = dat.obs.index.values.astype(int)

In [ ]:
bro.filters.objectfilterlib.write_filter_to_db(
    (dat.obs[[db.object_filters.object_id.key, 'mitosis']].rename(columns={'mitosis': db.object_filters.filter_value.key}))
    , V.FIL_MITOSIS, drop=True)

In [ ]:
bro.filters.objectfilterlib.write_filter_to_db(
    (dat.obs[[db.object_filters.object_id.key, 'dead']].rename(columns={'dead': db.object_filters.filter_value.key}))
    , V.FIL_APOPTOSIS , drop=True)

In [ ]:
sc.pl.umap(dat, color=list(dat.var.goodname)+['distrim','dead', 'mitosis'],gene_symbols='goodname')

Calculate MeanNbIntensity of all cells, excluding mitosis or apoptosis

In [ ]:
import sqlalchemy as sa

In [ ]:
fil_norm = bro.filters.objectfilterlib.get_combined_filterquery(
    object_filters=[(V.FIL_MITOSIS,0),(V.FIL_APOPTOSIS,0)])

In [ ]:
(bro.session.query(db.objects)
     .filter(db.objects.object_id==fil_norm.c.object_id)).count()

In [ ]:
bro.processing.nb_aggregation.add_nb_measurement('NbMeanNorm',np.mean, object_type='cell',
                                                 measurement_name='MeanIntensityComp',
                                                    stack_name='FullStackFiltered',
                                            filter_query=fil_norm,
                                            #remove_fil_nb=False,
                                               # drop_all_old=True
                                                )